In [1]:
import opal
import kglab
import rdflib
import z3
import dateutil.parser
import re
import numpy as np
import pandas as pd
import os
import sexpdata

In [2]:
from opal.logic.z3.z3_literal import Z3Literal
from opal.data.event_data import EventData
from opal.data.mapper import LogMapper
from opal.data.processor import LogProcessor

import opal.logic.z3.z3_literal as z3_literal
from opal.logic.z3.z3_ontology import Z3Ontology as Ontology
from opal.logic.z3.z3_reasoner import Z3Reasoner as Reasoner
from opal.logic.z3.z3_ontology import REF_ONTOLOGY_Z3, PSL_CORE_Z3, PSL_COMPLEX_Z3, PSL_OCCTREE_Z3, PSL_SUBACTIVITY_Z3

from rdflib import Graph

from z3 import *
z3.set_param(proof=True)
z3.set_param('verbose', 0)

Adding axiom: Reference Taxonomy_ont_t_0_disjoint_classes
Adding axiom: Reference Taxonomy_ont_t_1_event_order_SIMPLE
Adding axiom: Reference Taxonomy_ont_t_2_infer_next_event
Adding axiom: PSL Core Ontology_ont_t_0_type_disjointness
Adding axiom: PSL Core Ontology_ont_t_1_begin_unique
Adding axiom: PSL Core Ontology_ont_t_2_ends_unique
Adding axiom: PSL Core Ontology_ont_t_3_occurrence_bounds
Adding axiom: PSL Core Ontology_ont_t_4_occurrence_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_5_unique_activity_occurrence
Adding axiom: PSL Core Ontology_ont_t_6_occurrence_has_activity
Adding axiom: PSL Core Ontology_ont_t_7_participation_t_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_8_participation_time_to_generic
Adding axiom: PSL Core Ontology_ont_t_9_participation_sort_constraints
Adding axiom: PSL Core Ontology_ont_t_10_participation_temporal_constraint
Adding axiom: PSL Core Ontology_ont_t_11_object_temporal_existence
Adding axiom: PSL Core Ontology_ont_t_12_occurre

In [3]:
data_dir = os.path.abspath('../data/')
ttl_data_file = data_dir + '/P1_log_instances.ttl'
sample_log_file = data_dir + '/sample_log.csv'
bpic2017_file = data_dir + '/BPI Challenge 2017.xes.gz'

ontology_dir = os.path.abspath('../../opal/logic/z3/ontologies/')
psl_core_file = ontology_dir + '/PSL/PSL_core.smt2'
psl_core_mapping_file = ontology_dir + '/PSL/PSL_core_mapping.smt2'

## Mapping and reasoning on a simple XES log

### Mapping

In [4]:
# load the log
col_dict = {'caseID' : 'caseID', 'activityID' : 'activityID', 'timestamp' : 'timestamp', 'eventID' : None, 'transition' : 'transition'}
processor = LogProcessor(col_dict=col_dict, log_path=sample_log_file, batch_size=1)
event_data = processor.load_log()

In [5]:
# viewing the original data
event_data.data

,caseID,activityID,eventID,timestamp,resourceID,transition,processID,batch
0,C_case_0,A_activity_A,E_0,2016-01-01 09:00:00+00:00,user_1,T_start,P1,1
1,C_case_0,A_activity_A,E_1,2016-01-01 09:15:00+00:00,user_1,T_complete,P1,1
2,C_case_0,A_activity_C,E_2,2016-01-01 09:35:00+00:00,user_1,T_complete,P1,1
3,C_case_1,A_activity_A,E_3,2016-01-02 09:00:00+00:00,user_1,T_complete,P1,2
4,C_case_1,A_activity_B,E_4,2016-01-02 09:00:00+00:00,user_0,T_complete,P1,2
5,C_case_1,A_activity_C,E_5,2016-01-03 09:00:00+00:00,user_0,T_complete,P1,2
6,C_case_2,A_activity_A,E_6,2016-01-01 09:00:00+00:00,user_1,T_complete,P1,3
7,C_case_2,A_activity_B,E_7,2016-01-01 09:10:00+00:00,user_1,T_complete,P1,3
8,C_case_2,A_activity_C,E_8,2016-01-01 09:15:00+00:00,user_2,T_complete,P1,3
9,C_case_2,A_activity_D,E_9,2017-01-06 10:35:00+00:00,user_1,T_complete,P1,3


In [6]:
# metadata is automatically created
event_data.metadata

{'log_paths': ['/tmp/tmphzmjs2dp.csv',
  '/tmp/tmpwdetn7xf.csv',
  '/tmp/tmp8_vlg6mc.csv'],
 'process_name': 'P1',
 'batches': 3}

In [7]:
# initialize a mapper to map the log to logical facts according to the specified encoding (here, z3)
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/3 [00:00<?, ?it/s]2025-11-12 19:11:15,377 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-12 19:11:15,379 | INFO: RML content is created!
2025-11-12 19:11:15,388 | INFO: Mapping has been syntactically validated.
2025-11-12 19:11:15,389 | INFO: Translation has finished successfully.
2025-11-12 19:11:15,400 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-12 19:11:15,401 | INFO: RML content is created!
2025-11-12 19:11:15,408 | INFO: Mapping has been syntactically validated.
2025-11-12 19:11:15,409 | INFO: Translation has finished successfully.
2025-11-12 19:11:15,419 | INFO: Translating YARRRML mapping to [R2]RML
2025-11-12 19:11:15,420 | INFO: RML content is created!
2025-11-12 19:11:15,426 | INFO: Mapping has been syntactically validated.
2025-11-12 19:11:15,427 | INFO: Translation has finished successfully.
100%|██████████| 3/3 [00:00<00:00, 50.04it/s]

Mappings built.
Generating knowledge graph...



INFO | 2025-11-12 19:11:16,133 | 11 mapping rules retrieved.
INFO | 2025-11-12 19:11:16,148 | Mapping partition with 11 groups generated.
INFO | 2025-11-12 19:11:16,149 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-12 19:11:16,150 | Mappings processed in 0.714 seconds.
INFO | 2025-11-12 19:11:16,706 | Number of triples generated in total: 25.


Knowledge graph generated.
Knowledge graph 1/3 generated.
Generating knowledge graph...


INFO | 2025-11-12 19:11:17,304 | 11 mapping rules retrieved.
INFO | 2025-11-12 19:11:17,321 | Mapping partition with 11 groups generated.
INFO | 2025-11-12 19:11:17,323 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-12 19:11:17,323 | Mappings processed in 0.607 seconds.
INFO | 2025-11-12 19:11:17,700 | Number of triples generated in total: 26.


Knowledge graph generated.
Knowledge graph 2/3 generated.
Generating knowledge graph...


INFO | 2025-11-12 19:11:18,266 | 11 mapping rules retrieved.
INFO | 2025-11-12 19:11:18,278 | Mapping partition with 11 groups generated.
INFO | 2025-11-12 19:11:18,279 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-12 19:11:18,280 | Mappings processed in 0.572 seconds.
INFO | 2025-11-12 19:11:18,618 | Number of triples generated in total: 33.


Knowledge graph generated.
Knowledge graph 3/3 generated.
Z3 literal set (1/3) generated.
Z3 literal set (2/3) generated.
Z3 literal set (3/3) generated.


In [8]:
# rdf is still generated, because it is a prerequisite for mapping to more expressive encoding languages
graph = mapper.kgs[0].rdf_graph()
for triple in graph.triples((None, None, None)):
    print(triple)

(rdflib.term.URIRef('http://www.example.com/C_case_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasProcess'), rdflib.term.URIRef('http://www.example.com/P1'))
(rdflib.term.URIRef('http://www.example.com/E_0'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasLifecycleTransition'), rdflib.term.URIRef('http://www.example.com/T_start'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasCase'), rdflib.term.URIRef('http://www.example.com/C_case_0'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasActivity'), rdflib.term.URIRef('http://www.example.com/A_activity_A'))
(rdflib.term.URIRef('http://www.example.com/E_2'), rdflib.term.URIRef('https://stl.mie.utoronto.ca/ontologies/spm/hasLifecycleTransition'), rdflib.term.URIRef('http://www.example.com/T_complete'))
(rdflib.term.URIRef('http://www.example.com/E_1'), rdflib.t

### Reasoning Capabilities

In [9]:
# load the ontology (together with the mapping)
psl_core = Ontology.from_smt2(psl_core_file, psl_core_mapping_file)

Adding axiom: None_ont_t_0_type_disjointness
Adding axiom: None_ont_t_1_begin_unique
Adding axiom: None_ont_t_2_ends_unique
Adding axiom: None_ont_t_3_occurrence_bounds
Adding axiom: None_ont_t_4_occurrence_sort_constraints
Adding axiom: None_ont_t_5_unique_activity_occurrence
Adding axiom: None_ont_t_6_occurrence_has_activity
Adding axiom: None_ont_t_7_participation_t_sort_constraints
Adding axiom: None_ont_t_8_participation_time_to_generic
Adding axiom: None_ont_t_9_participation_sort_constraints
Adding axiom: None_ont_t_10_participation_temporal_constraint
Adding axiom: None_ont_t_11_object_temporal_existence
Adding axiom: None_ont_t_12_occurrence_temporal_extent
Adding axiom: None_map_t_0_occurrence_start_end_event_mapping
Adding axiom: None_map_t_1_occurrence_participation_mapping


In [10]:
psl_core.axioms

[{'name': 'None_ont_t_0_type_disjointness',
  'description': 'Activities, occurrences, and objects are different things.',
  'expr': '(assert (! (forall ((x Ind)) (and (=> (activity x) (not (or (activity_occurrence x) (object_ x)))) (=> (activity_occurrence x) (not (or (object_ x) (activity x)))) (=> (object_ x) (not (or (activity_occurrence x) (activity x)))))) :named type_disjointness :description "Activities, occurrences, and objects are different things."))'},
 {'name': 'None_ont_t_1_begin_unique',
  'description': 'Start points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (begin_of x) t1) (= (begin_of x) t2)) (= t1 t2))) :named begin_unique :description "Start points are unique."))'},
 {'name': 'None_ont_t_2_ends_unique',
  'description': 'End points are unique.',
  'expr': '(assert (! (forall ((x Ind) (t1 Real) (t2 Real)) (=> (and (= (end_of x) t1) (= (end_of x) t2)) (= t1 t2))) :named ends_unique :description "End points are unique."))'},

In [11]:
psl_core.literals

[]

In [12]:
z3_facts = []
for i, fact in enumerate(z3_data):
    prefix = f'mapped_a_{i}_'
    expr = fact
    name = prefix + f'{expr}'
    z3_facts.append({name : expr})

In [13]:
z3_facts[:5]

[{'mapped_a_0_hasProcess(C_case_0, P1)': <opal.logic.z3.z3_literal.Z3Literal at 0x70affbda8520>},
 {'mapped_a_1_hasLifecycleTransition(E_0, T_start)': <opal.logic.z3.z3_literal.Z3Literal at 0x70affbda2400>},
 {'mapped_a_2_hasCase(E_1, C_case_0)': <opal.logic.z3.z3_literal.Z3Literal at 0x70affbda2760>},
 {'mapped_a_3_hasActivity(E_1, A_activity_A)': <opal.logic.z3.z3_literal.Z3Literal at 0x70affbda25e0>},
 {'mapped_a_4_hasLifecycleTransition(E_2, T_complete)': <opal.logic.z3.z3_literal.Z3Literal at 0x70affbda23d0>}]

In [14]:
module = PSL_COMPLEX_Z3
r = Reasoner()
r.load_ontology(module)
r.load_data_declarations(mapper._z3_env)
r.load_facts(z3_facts)
r.add_relation_closure_axiom('hasCase')
r.add_relation_closure_axiom('Event')
r.add_relation_closure_axiom('Case')
r.add_unique_name_axioms('Event')
r.add_unique_name_axioms('Case')
s = r.check()

Loading assertion: Reference Taxonomy_ont_t_0_disjoint_classes : (assert (! (forall ((x Ind)) (and (=> (Event x) (not (or (Transition x) (Activity x) (Case x) (Resource x)))) (=> (Transition x) (not (or (Activity x) (Case x) (Resource x)))) (=> (Activity x) (not (or (Case x) (Resource x)))) (=> (Case x) (not (Resource x))))) :named disjoint_classes :description "Events, transitions, activities, cases, and resources are disjoint."))
Loading assertion: Reference Taxonomy_ont_t_1_event_order_SIMPLE : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (Event e1) (Event e2) (Case c) (hasCase e1 c) (hasCase e2 c) (< (hasRecordedTime e1) (hasRecordedTime e2))) (event_earlier e1 e2 c))) :named event_order_SIMPLE))
Loading assertion: Reference Taxonomy_ont_t_2_infer_next_event : (assert (! (forall ((e1 Ind) (e2 Ind) (c Ind)) (= (and (event_earlier e1 e2 c) (not (exists ((e3 Ind) (t3 Real)) (and (Event e3) (not (= e3 e1)) (not (= e3 e2)) (hasCase e3 c) (< (hasRecordedTime e3) (hasRecordedTim

In [15]:
# Since the data is consistent with the ontology, we have a satisfying interpretation and can extract a model
print(s.model())

[mapped_a_8_hasRecordedTime(E_1) == 1451639700 = True,
 PSL Occtree Ontology_ont_t_3_earlier_transitive = True,
 PSL Core Ontology_ont_t_6_occurrence_has_activity = True,
 mapped_a_47_Event(E_4) = True,
 mapped_a_74_hasCase(E_6, C_case_2) = True,
 E_8 = Ind!val!7,
 PSL Occtree Ontology_ont_t_1_earlier_requires_arboreal = True,
 PSL Complex Ontology_ont_t_8_act_tree_branch_consistency = True,
 A_activity_A = Ind!val!24,
 mapped_a_62_Transition(T_complete) = True,
 mapped_a_51_hasCase(E_8, C_case_2) = True,
 E_0 = Ind!val!6,
 Reference Taxonomy_ont_t_2_infer_next_event = True,
 PSL Complex Ontology_ont_t_5_root_legal = True,
 mapped_a_18_Resource(user_1) = True,
 mapped_a_26_hasProcess(C_case_1, P1) = True,
 mapped_a_72_hasActivity(E_7, A_activity_B) = True,
 mapped_a_12_hasCase(E_2, C_case_0) = True,
 Reference Taxonomy_ont_a_1_transition_complete = True,
 mapped_a_13_hasActivity(E_0, A_activity_A) = True,
 mapped_a_23_Transition(T_start) = True,
 mapped_a_63_hasActivity(E_6, A_activity

In [69]:
for d in m:
    if str(d) == 'activity_occurrence':
        x = d

In [ ]:
# get the individuals loaded into the reasoner
inds = set(r.decls.values()).difference(set(r.functions.values()))
for ind in inds:
    print(ind, m[ind])
# get the universe of individuals (includes inferred individuals)
universe = m.get_universe(r.sorts['Ind'])
universe_size = len(universe)
print("Universe size:", universe_size)
print('individual size:', len(inds))

# find difference between the two sets of individuals
ind_set = set()
for ind in inds:
    ind_set.add(m[ind])
print('mapped individual size:', len(ind_set))
diff = set(universe).difference(ind_set)
print('difference size:', len(diff))
diff

E_8 Ind!val!7
A_activity_A Ind!val!24
E_3 Ind!val!8
E_0 Ind!val!6
E_7 Ind!val!2
E_1 Ind!val!3
user_0 Ind!val!19
T_complete Ind!val!14
A_activity_C Ind!val!16
P1 Ind!val!15
E_5 Ind!val!0
C_case_0 Ind!val!12
user_1 Ind!val!17
C_case_2 Ind!val!11
E_4 Ind!val!9
A_activity_B Ind!val!18
E_6 Ind!val!5
T_start Ind!val!13
C_case_1 Ind!val!10
E_2 Ind!val!4
E_9 Ind!val!1
A_activity_D Ind!val!20
user_2 Ind!val!21
Universe size: 28
individual size: 23
mapped individual size: 23
difference size: 5


{Ind!val!22, Ind!val!23, Ind!val!25, Ind!val!26, Ind!val!27}

In [82]:
r.functions

{'Event': Event,
 'Transition': Transition,
 'Activity': Activity,
 'Case': Case,
 'Resource': Resource,
 'hasResource': hasResource,
 'hasRecordedTime': hasRecordedTime,
 'hasLifecycleTransition': hasLifecycleTransition,
 'hasCase': hasCase,
 'hasActivity': hasActivity,
 'next_event': next_event,
 'event_earlier': event_earlier,
 'transition': transition,
 'activity': activity,
 'activity_occurrence': activity_occurrence,
 'object_': object_,
 'timepoint': timepoint,
 'occurrence_of': occurrence_of,
 'participates_in': participates_in,
 'participates': participates,
 'exists_at': exists_at,
 'is_occurring_at': is_occurring_at,
 'begin_of': begin_of,
 'end_of': end_of,
 'ev_occ': ev_occ,
 'arboreal': arboreal,
 'earlier': earlier,
 'earlierEq': earlierEq,
 'initial': initial,
 'generator': generator,
 'legal': legal,
 'precedes': precedes,
 'poss': poss,
 'successor': successor,
 'subactivity': subactivity,
 'primitive': primitive,
 'hand_off': hand_off,
 'ping_pong': ping_pong,
 'min_

In [72]:
m[x].sexpr()

AttributeError: 'FuncInterp' object has no attribute 'sexpr'

In [16]:
m = r.model
m[r.decls['hasLifecycleTransition']]

[else ->
 And(Not(And(k!4612(Var(0)) == Ind!val!3,
             Var(1) == Ind!val!13)),
     Not(And(k!4612(Var(0)) == Ind!val!0,
             Var(1) == Ind!val!13)),
     Not(And(k!4612(Var(0)) == Ind!val!17,
             Var(1) == Ind!val!13)))]

In [52]:
m.get_universe(r.sorts['Ind'])[0]

Ind!val!27

In [55]:
m.get_interp(r.decls['hasLifecycleTransition'])

[else ->
 And(Not(And(k!4612(Var(0)) == Ind!val!3,
             Var(1) == Ind!val!13)),
     Not(And(k!4612(Var(0)) == Ind!val!0,
             Var(1) == Ind!val!13)),
     Not(And(k!4612(Var(0)) == Ind!val!17,
             Var(1) == Ind!val!13)))]

In [ ]:
universe = m.get_universe(r.sorts['Ind'])
for ind in universe:


    

In [ ]:
x = m[r.decls['hasLifecycleTransition']]
x.as_list()[0]

AttributeError: 'BoolRef' object has no attribute 'str'

In [ ]:
m[r.decls['occurrence_of']]

[else ->
 And(k!4613(Var(0)) == Ind!val!23,
     k!4613(Var(1)) == Ind!val!24)]

In [18]:
m[r.decls['activity_occurrence']]

[else -> k!4613(Var(0)) == Ind!val!23]

In [ ]:
m[r.decls['ev_occ']]

[else ->
 If(And(k!6021(Var(0)) == Ind!val!6,
        k!6021(Var(1)) == Ind!val!9),
    Ind!val!45,
    Ind!val!24)]

### Pattern Usage

In [ ]:
from opal.logic.z3.z3_pattern import HAND_OFF_PATTERN, PING_PONG_PATTERN, PING_PONG_QUERY_PATTERN
ping_pong_q = PING_PONG_QUERY_PATTERN.apply()
s = r.query(ping_pong_q)

Adding query: p_Hand-off_ : 
(assert (! (forall ((r1 Ind) (r2 Ind) (o1 Ind) (o2 Ind) (c Ind))
    (= (and
            (next_subocc o1 o2 c)
            (participates r1 o1)
            (participates r2 o2)
            (distinct r1 r2)
        )
    (hand_off r1 r2 o1 o2 c))
)))
Adding query: p_Ping-pong_ : 
(assert (! (forall ((c Ind))
    (= (exists ((e1 Ind) (e2 Ind) (e3 Ind) (e4 Ind) (r1 Ind) (r2 Ind) (r3 Ind) (r4 Ind))
            (and
                (hand_off r1 r2 e1 e2 c)
                (hand_off r3 r4 e3 e4 c)
                (distinct e1 e3)
                (distinct e2 e4)
            ))
        (ping_pong c)
    )
)))
Adding query: p_Ping-pong Query_ : 
(assert (! (not (exists ((c Ind))
    (ping_pong c))
)))


In [ ]:
r.add_relation_closure_axiom('activity_occurrence')

No instances found for relation activity_occurrence. Skipping closure.


In [ ]:
s = r.check()

: 

In [ ]:
m = s.model()

In [20]:
m[r.decls['min_precedes']]

[else -> False]

# Mapping BPIC 2017 data

In [ ]:
processor = LogProcessor(log_path=bpic2017_file, batch_size=100)
event_data = processor.load_log(downsample_rate=0.05)

/home/vscode/.local/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:82: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:57<00:00, 546.80it/s]
/workspaces/ProMean Lib/opal/data/processor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['batch'] = (df.groupby('caseID').ngroup() // self.batch_size) + 1


In [ ]:
mapper = LogMapper(data=event_data, output_encoding='z3')
z3_data = mapper.get_mapped_data()

Building mappings for logs...


  0%|          | 0/16 [00:00<?, ?it/s]2025-08-11 14:02:34,488 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,488 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,490 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,490 | RML content is created!
2025-08-11 14:02:34,496 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,496 | Mapping has been syntactically validated.
2025-08-11 14:02:34,498 | INFO: Translation has finished successfully.
INFO | 2025-08-11 14:02:34,498 | Translation has finished successfully.
2025-08-11 14:02:34,508 | INFO: Translating YARRRML mapping to [R2]RML
INFO | 2025-08-11 14:02:34,508 | Translating YARRRML mapping to [R2]RML
2025-08-11 14:02:34,509 | INFO: RML content is created!
INFO | 2025-08-11 14:02:34,509 | RML content is created!
2025-08-11 14:02:34,516 | INFO: Mapping has been syntactically validated.
INFO | 2025-08-11 14:02:34,516 | Mapping has been syntactically validated.
2025-08-1

Mappings built.
Generating knowledge graph...


INFO | 2025-08-11 14:02:35,287 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:02:35,297 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:02:35,299 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:02:35,299 | Mappings processed in 0.519 seconds.
INFO | 2025-08-11 14:02:36,870 | Number of triples generated in total: 23614.


Knowledge graph generated.
Knowledge graph 1/16 generated.
Generating knowledge graph...


...

Knowledge graph generated.
Knowledge graph 14/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:11,740 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:11,754 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:11,755 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:11,756 | Mappings processed in 0.540 seconds.
INFO | 2025-08-11 14:03:13,130 | Number of triples generated in total: 22243.


Knowledge graph generated.
Knowledge graph 15/16 generated.
Generating knowledge graph...


INFO | 2025-08-11 14:03:15,046 | 11 mapping rules retrieved.
INFO | 2025-08-11 14:03:15,057 | Mapping partition with 11 groups generated.
INFO | 2025-08-11 14:03:15,058 | Maximum number of rules within mapping group: 1.
INFO | 2025-08-11 14:03:15,059 | Mappings processed in 0.538 seconds.
INFO | 2025-08-11 14:03:16,380 | Number of triples generated in total: 17015.


Knowledge graph generated.
Knowledge graph 16/16 generated.
Z3 literal set generated (1/16)
Z3 literal set generated (2/16)
Z3 literal set generated (3/16)
Z3 literal set generated (4/16)
Z3 literal set generated (5/16)
Z3 literal set generated (6/16)
Z3 literal set generated (7/16)
Z3 literal set generated (8/16)
Z3 literal set generated (9/16)
Z3 literal set generated (10/16)
Z3 literal set generated (11/16)
Z3 literal set generated (12/16)
Z3 literal set generated (13/16)
Z3 literal set generated (14/16)
Z3 literal set generated (15/16)
Z3 literal set generated (16/16)


In [ ]:
len(z3_data)

359577

In [ ]:
len(mapper.data.data)

59801

In [ ]:
str(z3_data[5008])

'hasResource(E_1174728, R_User_29)'